In [7]:
# Build lookup table to be used in c++ code
# Speeds going from 0,5 cm/m till 30cm/min

import numpy as np
# first the interval between 0.5 till 0.99
speeds = np.arange(0.5, 1.0, 0.01)

# next interval is 1.0 till 4.9
next_interval = np.arange(1.0, 5.0, 0.1)
speeds = np.concatenate((speeds,next_interval))

# next interval is 5.0 till 16
next_interval = np.arange(5, 17, 1)
speeds = np.concatenate((speeds,next_interval))

# fixed this line to start at 18 - previously 16 was repeated.
# next interval is 16 (previously added) to 22 in increments of 2
next_interval = np.arange(18, 24, 2)
speeds = np.concatenate((speeds,next_interval))

# next interval is 24 to 30 (inclusive) in increments of 3
next_interval = np.arange(24, 31, 3)
speeds = np.concatenate((speeds,next_interval))
speeds

def uSPulse(mmT, sR, d):
    # micro seconds per pulse (logic high, equal in width to logic low - room for improv. shorter pulse, lesser power required?)
    St = (sR * d) / mmT
    #print(St)
    uSm = 60000000 # microseconds per minute
    P = uSm / St
    hP = P / 2
    return int(hP)

# constants using a lead screw
# mmT = 1.25 # mm per thread
# sR = 200 # steps per revolution

# Timing belt GT2 2mm/thread.
# 18-teeth gear attached to steppert motor shaft
# driving 48-teeth gear with 16-teeth gear in the middle,
# attached to a 60-teeth gear attached to a 20-teeth gear in the middle,
# attached to GT2 timing belt.
# 48/18 = 2.6667 stepper motor revolutions (18) per 1 48-teeth gear revolution
# 60/16 = 3.75 16-teeth gear revolutions per 1 60-teeth gear revolution
# 1 20-teeth gear revolution = 40mm linear dislocation of timing belt
# 2.6667 * 3.75 = 10.000125 stepper motor revolution correspond to 40mm 
# linear dislocation.
# 1 stepper motor revolution = 4m linear dislocation
# Stepper motor set up to 400 steps/revolution
# mmT = 4
# sR = 400
# to get sR = 200, we divide mmT/2 = 2mm linear dislocation

mmT = 2 
sR = 50 # steps per revolution, GT2 timing belt

# our c++ comments and array declarations which we paste into our sketch
# declaration strings
strdec = "char *speeds[] = { "
# declaration integers
strdec2 = "unsigned int halffreq[] = { "

for i in range(0,speeds.size):
    # will need distance in milimetres so multiply by 100
    d = speeds[i] * 10 # distance to be covered in mm over the course of one minute
    pm = uSPulse(mmT, sR, d) # pulses per minute
    # format strings
    strval = "{:10.2f}".format(speeds[i])[5:]
    strval = "\"" + strval + "\", "
    strdec += strval
    # format integers
    strdec2 += str(pm) + ", "

# Speed nominals
# remove last comma
strdec = strdec[:-2]
# and add closing bracket plus semicolon
strdec += " };"
# print comment
print("// Nominal speeds array")
print(strdec)

# Speed half frequencies
# Speed nominals
# remove last comma
strdec2 = strdec2[:-2]
# and add closing bracket plus semicolon
strdec2 += " };"
# print comment
print("// Half frequencies array")
print(strdec2)

# print array info
print("// array boundaries")
print("int arrMin = 0;")
# subtract one because of 0 based index
print("int arrMax = " + str(speeds.size - 1) + ";")

// Nominal speeds array
char *speeds[] = { " 0.50", " 0.51", " 0.52", " 0.53", " 0.54", " 0.55", " 0.56", " 0.57", " 0.58", " 0.59", " 0.60", " 0.61", " 0.62", " 0.63", " 0.64", " 0.65", " 0.66", " 0.67", " 0.68", " 0.69", " 0.70", " 0.71", " 0.72", " 0.73", " 0.74", " 0.75", " 0.76", " 0.77", " 0.78", " 0.79", " 0.80", " 0.81", " 0.82", " 0.83", " 0.84", " 0.85", " 0.86", " 0.87", " 0.88", " 0.89", " 0.90", " 0.91", " 0.92", " 0.93", " 0.94", " 0.95", " 0.96", " 0.97", " 0.98", " 0.99", " 1.00", " 1.10", " 1.20", " 1.30", " 1.40", " 1.50", " 1.60", " 1.70", " 1.80", " 1.90", " 2.00", " 2.10", " 2.20", " 2.30", " 2.40", " 2.50", " 2.60", " 2.70", " 2.80", " 2.90", " 3.00", " 3.10", " 3.20", " 3.30", " 3.40", " 3.50", " 3.60", " 3.70", " 3.80", " 3.90", " 4.00", " 4.10", " 4.20", " 4.30", " 4.40", " 4.50", " 4.60", " 4.70", " 4.80", " 4.90", " 5.00", " 6.00", " 7.00", " 8.00", " 9.00", "10.00", "11.00", "12.00", "13.00", "14.00", "15.00", "16.00", "18.00", "20.00", "22.00", "24.00", "27

Previous approach did not scale well as array variables are stored in dynamic memory.
So we'll use program memory instead to store our lookup tables

### PROGMEM ALTERNATIVE VARIABLES

In [21]:
# New PROGMEM construct
# print literal speeds
for i in range(0,speeds.size):
    # will need distance in milimetres so multiply by 100
    d = speeds[i] * 10 # distance to be covered in mm over the course of one minute
    pm = uSPulse(mmT, sR, d) # pulses per minute
    # format strings
    strval = "{:10.2f}".format(speeds[i])[5:]
    strval = "const char speed_" + str(i) + "[] PROGMEM = \"" + strval + "\"; "
    print(strval)


const char speed_0[] PROGMEM = " 0.50"; 
const char speed_1[] PROGMEM = " 0.51"; 
const char speed_2[] PROGMEM = " 0.52"; 
const char speed_3[] PROGMEM = " 0.53"; 
const char speed_4[] PROGMEM = " 0.54"; 
const char speed_5[] PROGMEM = " 0.55"; 
const char speed_6[] PROGMEM = " 0.56"; 
const char speed_7[] PROGMEM = " 0.57"; 
const char speed_8[] PROGMEM = " 0.58"; 
const char speed_9[] PROGMEM = " 0.59"; 
const char speed_10[] PROGMEM = " 0.60"; 
const char speed_11[] PROGMEM = " 0.61"; 
const char speed_12[] PROGMEM = " 0.62"; 
const char speed_13[] PROGMEM = " 0.63"; 
const char speed_14[] PROGMEM = " 0.64"; 
const char speed_15[] PROGMEM = " 0.65"; 
const char speed_16[] PROGMEM = " 0.66"; 
const char speed_17[] PROGMEM = " 0.67"; 
const char speed_18[] PROGMEM = " 0.68"; 
const char speed_19[] PROGMEM = " 0.69"; 
const char speed_20[] PROGMEM = " 0.70"; 
const char speed_21[] PROGMEM = " 0.71"; 
const char speed_22[] PROGMEM = " 0.72"; 
const char speed_23[] PROGMEM = " 0.73"; 
co

const char * const string_table[] PROGMEM =	   // change "string_table" name to suit
{   
  string_0,
  string_1,
  string_2,
  string_3,
  string_4,
  string_5 };


In [22]:
# print string table
# print literal speeds
print("const char * const speed_table[] PROGMEM = { ")
for i in range(0,speeds.size):
    # will need distance in milimetres so multiply by 100
    d = speeds[i] * 10 # distance to be covered in mm over the course of one minute
    pm = uSPulse(mmT, sR, d) # pulses per minute
    # format strings
    strval = "{:10.2f}".format(speeds[i])[5:]
    strval = "    speed_" + str(i) + ", "
    print(strval)
    strdec += strval
    # format integers
    strdec2 += str(pm) + ", "
print("};")

const char * const speed_table[] PROGMEM = { 
    speed_0, 
    speed_1, 
    speed_2, 
    speed_3, 
    speed_4, 
    speed_5, 
    speed_6, 
    speed_7, 
    speed_8, 
    speed_9, 
    speed_10, 
    speed_11, 
    speed_12, 
    speed_13, 
    speed_14, 
    speed_15, 
    speed_16, 
    speed_17, 
    speed_18, 
    speed_19, 
    speed_20, 
    speed_21, 
    speed_22, 
    speed_23, 
    speed_24, 
    speed_25, 
    speed_26, 
    speed_27, 
    speed_28, 
    speed_29, 
    speed_30, 
    speed_31, 
    speed_32, 
    speed_33, 
    speed_34, 
    speed_35, 
    speed_36, 
    speed_37, 
    speed_38, 
    speed_39, 
    speed_40, 
    speed_41, 
    speed_42, 
    speed_43, 
    speed_44, 
    speed_45, 
    speed_46, 
    speed_47, 
    speed_48, 
    speed_49, 
    speed_50, 
    speed_51, 
    speed_52, 
    speed_53, 
    speed_54, 
    speed_55, 
    speed_56, 
    speed_57, 
    speed_58, 
    speed_59, 
    speed_60, 
    speed_61, 
    speed_62, 
    speed_63, 
    

// save some unsigned ints
```
const uint16_t charSet[]  PROGMEM  = { 65000, 32796, 16843, 10, 11234};
```

In [8]:
print("const uint16_t halfFrequencies[]  PROGMEM  = { ")
for i in range(0,speeds.size):
    # will need distance in milimetres so multiply by 100
    d = speeds[i] * 10 # distance to be covered in mm over the course of one minute
    pm = uSPulse(mmT, sR, d) # pulses per minute
    # format strings
    #strval = "{:10.2f}".format(speeds[i])[5:]
    strval = "    " + str(pm) + ", "
    print(strval)
    #strdec += strval
    # format integers
    #strdec2 += str(pm) + ", "
print("};")    

const uint16_t halfFrequencies[]  PROGMEM  = { 
    240000, 
    235294, 
    230769, 
    226415, 
    222222, 
    218181, 
    214285, 
    210526, 
    206896, 
    203389, 
    199999, 
    196721, 
    193548, 
    190476, 
    187499, 
    184615, 
    181818, 
    179104, 
    176470, 
    173913, 
    171428, 
    169014, 
    166666, 
    164383, 
    162162, 
    159999, 
    157894, 
    155844, 
    153846, 
    151898, 
    149999, 
    148148, 
    146341, 
    144578, 
    142857, 
    141176, 
    139534, 
    137931, 
    136363, 
    134831, 
    133333, 
    131868, 
    130434, 
    129032, 
    127659, 
    126315, 
    124999, 
    123711, 
    122448, 
    121212, 
    120000, 
    109090, 
    99999, 
    92307, 
    85714, 
    79999, 
    74999, 
    70588, 
    66666, 
    63157, 
    59999, 
    57142, 
    54545, 
    52173, 
    49999, 
    47999, 
    46153, 
    44444, 
    42857, 
    41379, 
    39999, 
    38709, 
    37499, 
    36363, 
    35294, 


In [4]:
# Generate distance look-up table - to keep track of start and end of programmable course
i = 0.0
# while(i < 30.1):
while(i < 52.0):
    strval = "{:4.1f}".format(i)
    strarrval = strval
    # prepare string 
    strarrval = strarrval.lstrip()
    strarrval = strarrval.replace('.', '_')
    strval = "const char distance_" + strarrval + "[] PROGMEM = \"" + strval + "\"; "
    print(strval)
    i += 0.1

const char distance_0_0[] PROGMEM = " 0.0"; 
const char distance_0_1[] PROGMEM = " 0.1"; 
const char distance_0_2[] PROGMEM = " 0.2"; 
const char distance_0_3[] PROGMEM = " 0.3"; 
const char distance_0_4[] PROGMEM = " 0.4"; 
const char distance_0_5[] PROGMEM = " 0.5"; 
const char distance_0_6[] PROGMEM = " 0.6"; 
const char distance_0_7[] PROGMEM = " 0.7"; 
const char distance_0_8[] PROGMEM = " 0.8"; 
const char distance_0_9[] PROGMEM = " 0.9"; 
const char distance_1_0[] PROGMEM = " 1.0"; 
const char distance_1_1[] PROGMEM = " 1.1"; 
const char distance_1_2[] PROGMEM = " 1.2"; 
const char distance_1_3[] PROGMEM = " 1.3"; 
const char distance_1_4[] PROGMEM = " 1.4"; 
const char distance_1_5[] PROGMEM = " 1.5"; 
const char distance_1_6[] PROGMEM = " 1.6"; 
const char distance_1_7[] PROGMEM = " 1.7"; 
const char distance_1_8[] PROGMEM = " 1.8"; 
const char distance_1_9[] PROGMEM = " 1.9"; 
const char distance_2_0[] PROGMEM = " 2.0"; 
const char distance_2_1[] PROGMEM = " 2.1"; 
const char

In [5]:
# generate distance array reference
print("const char * const distance_table[] PROGMEM = { ")
i = 0.0
# while(i < 30.1):
while(i < 52.0):
    strval = "{:2.1f}".format(i)
    strarrval = strval
    # prepare string 
    strarrval = strarrval.lstrip()
    strarrval = strarrval.replace('.', '_')
    strval = "    distance_" + strarrval + ","
    print(strval)
    i += 0.1
print("};")    

const char * const distance_table[] PROGMEM = { 
    distance_0_0,
    distance_0_1,
    distance_0_2,
    distance_0_3,
    distance_0_4,
    distance_0_5,
    distance_0_6,
    distance_0_7,
    distance_0_8,
    distance_0_9,
    distance_1_0,
    distance_1_1,
    distance_1_2,
    distance_1_3,
    distance_1_4,
    distance_1_5,
    distance_1_6,
    distance_1_7,
    distance_1_8,
    distance_1_9,
    distance_2_0,
    distance_2_1,
    distance_2_2,
    distance_2_3,
    distance_2_4,
    distance_2_5,
    distance_2_6,
    distance_2_7,
    distance_2_8,
    distance_2_9,
    distance_3_0,
    distance_3_1,
    distance_3_2,
    distance_3_3,
    distance_3_4,
    distance_3_5,
    distance_3_6,
    distance_3_7,
    distance_3_8,
    distance_3_9,
    distance_4_0,
    distance_4_1,
    distance_4_2,
    distance_4_3,
    distance_4_4,
    distance_4_5,
    distance_4_6,
    distance_4_7,
    distance_4_8,
    distance_4_9,
    distance_5_0,
    distance_5_1,
    distance_5_